<a href="https://colab.research.google.com/github/Amal237985/Data_Analysis/blob/main/Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Text Analysis***

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import statistics as stat
import glob

In [ ]:
input = '/content/Input.xlsx'
df = pd.read_excel(input)
df

,URL_ID,URL
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...
3,bctech2014,https://insights.blackcoffer.com/effective-man...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...
...,...,...
142,bctech2153,https://insights.blackcoffer.com/population-an...
143,bctech2154,https://insights.blackcoffer.com/google-lsa-ap...
144,bctech2155,https://insights.blackcoffer.com/healthcare-da...
145,bctech2156,https://insights.blackcoffer.com/budget-sales-...


In [ ]:
output_dir = 'files'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1')
    title_text = title.get_text(strip=True) if title else 'No Title'
    article_text = ''
    article_tag = soup.find('div', class_='td-post-content')

    if article_tag:
        paragraphs = article_tag.find_all('p')
        article_text = '\n\n'.join([p.get_text(strip=True) for p in paragraphs])

    return title_text, article_text

In [ ]:
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    title, article_text = extract_text(url)

    with open(os.path.join(output_dir, f'{url_id}.txt'), 'w', encoding='utf-8') as file:
            file.write(f"Title: {title}\n\n")
            file.write(article_text)

    print(f"Successfully saved article {url_id}")



Successfully saved article bctech2011
Successfully saved article bctech2012
Successfully saved article bctech2013
Successfully saved article bctech2014
Successfully saved article bctech2015
Successfully saved article bctech2016
Successfully saved article bctech2017
Successfully saved article bctech2018
Successfully saved article bctech2019
Successfully saved article bctech2020
Successfully saved article bctech2021
Successfully saved article bctech2022
Successfully saved article bctech2023
Successfully saved article bctech2024
Successfully saved article bctech2025
Successfully saved article bctech2026
Successfully saved article bctech2027
Successfully saved article bctech2028
Successfully saved article bctech2029
Successfully saved article bctech2030
Successfully saved article bctech2031
Successfully saved article bctech2032
Successfully saved article bctech2033
Successfully saved article bctech2034
Successfully saved article bctech2035
Successfully saved article bctech2036
Successfully

In [ ]:
import os
import nltk
import chardet
import pandas as pd
import glob
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def detect_encoding(filepath):
    with open(filepath, 'rb') as file:
        raw_data = file.read()
    result = chardet.detect(raw_data)
    return result['encoding']

In [ ]:
def load_words_from_file(filepath):
    words = set()
    encoding=detect_encoding(filepath)
    with open(filepath, 'r',encoding=encoding) as file:
      for line in file:
          words.update(line.lower().strip().split())
    return words

In [ ]:
stop_words_1 = load_words_from_file('/content/StopWords_Auditor.txt')
stop_words_2 = load_words_from_file('/content/StopWords_Currencies.txt')
stop_words_3 = load_words_from_file('/content/StopWords_DatesandNumbers.txt')
stop_words_4 = load_words_from_file('/content/StopWords_Generic.txt')

stop_words=stop_words_1.union(stop_words_2).union(stop_words_3).union(stop_words_4)

positive_words = load_words_from_file('/content/positive-words.txt')
negative_words = load_words_from_file('/content/negative-words.txt')

print(f'Stop words : {stop_words}')
print(f'Positive words : {positive_words}')
print(f'Negative words : {negative_words}')


Stop words : {'now', 'third', 'our', 'lebanon', 'czech', 'lilangeni', 'under', 'my', 'rupiah', 'lira', 'tenge', 'vatu', 'september', 'ten', 'forint', 'by', 'kwacha', 'malaysia', 'two', 'because', 'those', 'xv', 'euro', 'xx', 'lari', 'so', 'madagascar', 'lempira', 'drawing', '(former', 'tugrik', 'been', 'coopers', 'annually', 'twenty', 'week', 'macau', 'rufiyaa', 'xii', 'som', 'eighteen', 'denominations', 'netherlands', 'yours', 'nicaragua', 'real', 'are', 'of', 'kwanza', 'xi', 'lesotho', 'kyat', 'among', 'aug', 'daily', 'we', 'rep.)', 'mongolia', 'ukraine', 'thailand', 'v', 'seventh', 'vi', 'belgium', 'nine', 'between', 'ouguiya', 'tala', 'viii', 'hundred', 'oct', 'kpmg', 'am', 'sweden', 'mexico', 'feb', 'rand', 'when', 'xiv', 'numerals', 'iran', 'balboa', 'baht', 'colon', 'sixth', 'pricewaterhouse', 'on', 'april', 'eighty', 'sixteen', 'what', 'ghana', 'january', 'lek', 'quarterly', 'verde', 'they', 'poland', 'saturday', 'during', 'these', 'israel', 'peso', 'maldives', 'qtr', 'once', '

In [ ]:
def tokenize(text):
    t = word_tokenize(text.lower())
    tokens = [word for word in t if word.isalpha() and word not in stop_words]
    return tokens

In [ ]:
def sentiment_scores(tokens):
    positive_score = 0
    negative_score = 0
    for word in tokens:
      if word in positive_words:
        positive_score += 1
      elif word in negative_words:
        negative_score += 1
    return positive_score, negative_score


In [ ]:
def read_text(filepath):
    encoding=detect_encoding(filepath)
    with open(filepath, 'r',encoding=encoding) as file:
        text = file.read()
    return text

In [ ]:
folder_path='/content/files'
file_paths = sorted(glob.glob(os.path.join(folder_path, '*.txt')))
positive_scores=[]
negative_scores=[]
for file_path in file_paths:
  text = read_text(file_path)
  tokens = tokenize(text)
  positive_score, negative_score = sentiment_scores(tokens)
  positive_scores.append(positive_score)
  negative_scores.append(negative_score)
print(f"Positive Score: {positive_scores}")
print(f"Negative Score: {negative_scores}")

Positive Score: [32, 2, 2, 2, 3, 2, 2, 1, 1, 2, 3, 17, 17, 4, 17, 31, 17, 21, 31, 26, 23, 3, 10, 20, 15, 28, 21, 11, 7, 5, 5, 5, 6, 10, 6, 5, 8, 6, 12, 10, 11, 13, 6, 13, 19, 6, 7, 5, 7, 6, 9, 13, 6, 5, 10, 12, 15, 13, 7, 6, 6, 8, 7, 6, 13, 12, 6, 22, 7, 9, 25, 6, 9, 2, 8, 8, 2, 2, 2, 2, 2, 11, 8, 13, 2, 3, 3, 5, 2, 7, 1, 24, 22, 1, 8, 6, 4, 3, 7, 5, 6, 1, 4, 1, 24, 13, 2, 6, 13, 8, 13, 9, 20, 1, 6, 1, 9, 5, 0, 3, 7, 4, 4, 3, 10, 1, 3, 10, 13, 9, 6, 4, 3, 2, 4, 4, 3, 2, 4, 4, 15, 9, 14, 22, 8, 0, 2]
Negative Score: [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 3, 3, 5, 3, 2, 8, 8, 12, 3, 0, 7, 0, 6, 11, 15, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 4, 2, 2, 1, 1, 4, 8, 4, 2, 1, 3, 1, 2, 1, 2, 4, 3, 8, 4, 5, 5, 1, 4, 4, 1, 8, 2, 4, 5, 13, 1, 0, 4, 2, 0, 1, 2, 2, 0, 7, 12, 0, 0, 1, 6, 3, 0, 0, 2, 2, 1, 1, 8, 2, 1, 0, 3, 3, 5, 1, 2, 0, 3, 1, 2, 0, 3, 0, 2, 0, 4, 3, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 3, 1, 0, 0, 8, 3, 2, 3, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 10, 7, 9, 19, 7, 0, 0]


In [ ]:
polarity_score=[]
for i in range(len(positive_scores)):
  polarity_score.append((positive_scores[i]-negative_scores[i])/ ((positive_scores[i] + negative_scores[i]) + 0.000001)) # Use positive_scores instead of positive_score
print(f"Polarity Score: {polarity_score}")

Polarity Score: [0.5999999850000004, 0.99999950000025, 0.99999950000025, 0.99999950000025, 0.9999996666667778, 0.99999950000025, 0.99999950000025, 0.9999990000010001, 0.9999990000010001, 0.99999950000025, 0.9999996666667778, 0.6190475895691624, 0.6999999650000017, 0.1428571224489825, 0.5454545206611581, 0.8235293875432533, 0.7894736426592819, 0.4482758466111777, 0.5897435746219597, 0.36842104293628836, 0.7692307396449716, 0.9999996666667778, 0.17647057785467188, 0.9999999500000024, 0.4285714081632663, 0.4358974247205789, 0.16666666203703717, 0.8333332638888947, 0.7499999062500118, 0.6666665555555741, 0.6666665555555741, 0.6666665555555741, 0.7142856122449125, 0.6666666111111158, 0.7142856122449125, 0.6666665555555741, 0.599999940000006, 0.4999999375000079, 0.4999999687500019, 0.6666666111111158, 0.6923076390532585, 0.8571427959183717, 0.7142856122449125, 0.5294117335640156, 0.4074073923182447, 0.19999998000000202, 0.5555554938271674, 0.6666665555555741, 0.39999996000000404, 0.714285612

In [ ]:
subjectivity_score=[]
for i in range(len(positive_scores)):
  subjectivity_score.append((positive_scores[i] + negative_scores[i])/ ((len(tokenize(read_text(file_paths[i])))) + 0.000001)) # Use positive_scores and negative_scores lists
print(f"Subjectivity Score: {subjectivity_score}")

Subjectivity Score: [0.08528784630002592, 0.03636363570247935, 0.03773584834460664, 0.03636363570247935, 0.05454545355371903, 0.03846153772189351, 0.03773584834460664, 0.018181817851239675, 0.015873015621063243, 0.03636363570247935, 0.049999999166666684, 0.0548302870631063, 0.07633587757123711, 0.04794520515106024, 0.058981233085841196, 0.10624999966796875, 0.06761565812236421, 0.07090464530341163, 0.11079545423069473, 0.10270270242512783, 0.06295399500495401, 0.02127659559378301, 0.06367041174655276, 0.0975609751338489, 0.0766423354867068, 0.0977443606572823, 0.07331975545148726, 0.06703910577073126, 0.05714285673469388, 0.05172413748513675, 0.11999999760000006, 0.06249999934895834, 0.04458598697715932, 0.05194805172316861, 0.04320987627648225, 0.05555555504115227, 0.06493506451340868, 0.06451612851196671, 0.06349206324011086, 0.06976744145484046, 0.0653266328375546, 0.09271523117407132, 0.04046242751177788, 0.08292682886377156, 0.1261682237094943, 0.040650406338819486, 0.057324840399

In [ ]:
average_sentence_lengths = []
for file_path in file_paths:
    text = read_text(file_path)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    num_sentences = len(sentences)
    num_words = len([word for word in words if word.isalpha()])
    Average_Sentence_Length = num_words/ num_sentences
    average_sentence_lengths.append(Average_Sentence_Length)

print(f"Average Sentence Lengths: {average_sentence_lengths}")

Average Sentence Lengths: [34.3, 23.666666666666668, 23.666666666666668, 24.333333333333332, 23.666666666666668, 23.333333333333332, 23.666666666666668, 23.333333333333332, 26.666666666666668, 25.333333333333332, 24.666666666666668, 16.61764705882353, 22.352941176470587, 27.0, 18.516129032258064, 22.045454545454547, 19.90909090909091, 19.967741935483872, 22.625, 18.161290322580644, 19.0625, 24.555555555555557, 16.130434782608695, 20.666666666666668, 18.08695652173913, 16.86842105263158, 33.12, 24.153846153846153, 18.916666666666668, 88.0, 34.0, 50.666666666666664, 67.0, 13.44, 21.083333333333332, 16.666666666666668, 42.2, 34.333333333333336, 27.8, 28.5, 34.77777777777778, 21.181818181818183, 23.454545454545453, 16.130434782608695, 15.538461538461538, 15.954545454545455, 25.77777777777778, 55.0, 20.0, 27.0, 24.375, 21.714285714285715, 18.166666666666668, 17.65, 27.846153846153847, 28.428571428571427, 26.214285714285715, 17.272727272727273, 16.2, 19.6, 23.6, 19.857142857142858, 19.6, 15.

In [ ]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiou"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [ ]:
percentage_complex_words_list = []
for file_path in file_paths:
    text = read_text(file_path)
    words = word_tokenize(text)
    num_words = len([word for word in words if word.isalpha()])

    cmplx_words = []
    for word in words:
        if word.isalpha() and syllable_count(word) > 2:
            cmplx_words.append(word)

    percentage_complex = len(cmplx_words) / num_words if num_words > 0 else 0 # Handle potential division by zero
    percentage_complex_words_list.append(percentage_complex)

print(f"Percentage of Complex Words for each file: {percentage_complex_words_list}")

Percentage of Complex Words for each file: [0.29591836734693877, 0.23943661971830985, 0.29577464788732394, 0.2876712328767123, 0.30985915492957744, 0.2857142857142857, 0.2535211267605634, 0.2714285714285714, 0.2875, 0.27631578947368424, 0.3108108108108108, 0.2176991150442478, 0.23421052631578948, 0.2037037037037037, 0.20034843205574912, 0.24948453608247423, 0.23972602739726026, 0.21647819063004847, 0.24125230202578268, 0.20781527531083482, 0.26885245901639343, 0.18552036199095023, 0.2991913746630728, 0.25161290322580643, 0.28846153846153844, 0.21060842433697347, 0.15096618357487923, 0.1305732484076433, 0.15859030837004406, 0.125, 0.17647058823529413, 0.14473684210526316, 0.13930348258706468, 0.19345238095238096, 0.18181818181818182, 0.17333333333333334, 0.24170616113744076, 0.13106796116504854, 0.15827338129496402, 0.1649122807017544, 0.20447284345047922, 0.12446351931330472, 0.1511627906976744, 0.09164420485175202, 0.12128712871287128, 0.22792022792022792, 0.16379310344827586, 0.18181

In [ ]:
fog_indices = []
for i in range(len(positive_scores)):
    fog_indices.append(0.4 * (average_sentence_lengths[i] + percentage_complex_words_list[i]))
print(f"Fog Indices for each file: {fog_indices}")

Fog Indices for each file: [13.838367346938774, 9.562441314553991, 9.584976525821597, 9.848401826484018, 9.590610328638498, 9.447619047619048, 9.568075117370894, 9.441904761904762, 10.781666666666668, 10.243859649122808, 9.990990990990992, 6.734138469547111, 9.03486068111455, 10.881481481481481, 7.486590985725525, 8.917975632614809, 8.059526774595268, 8.073688050445568, 9.146500920810313, 7.347642239156592, 7.732540983606558, 9.896430367018603, 6.571850462908708, 8.36731182795699, 7.350167224080268, 6.831611790787422, 13.308386473429952, 9.713767760901518, 7.630102790014686, 35.25, 13.670588235294119, 20.32456140350877, 26.855721393034827, 5.453380952380953, 8.506060606060606, 6.736000000000001, 16.976682464454978, 13.785760517799355, 11.183309352517988, 11.465964912280702, 13.992900248491305, 8.522512680452596, 9.442283298097252, 6.488831594984179, 6.2638994668697645, 6.472986272986273, 10.376628352490423, 22.072727272727274, 8.076, 10.896296296296297, 9.848461538461539, 8.73439849624

In [ ]:
average_words_per_sentence_list = []
for file_path in file_paths:
    text = read_text(file_path)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    num_sentences = len(sentences)
    num_words = len([word for word in words if word.isalpha()])
    average_words_per_sentence = num_words / num_sentences
    average_words_per_sentence_list.append(average_words_per_sentence)

print(f"Average Number of Words Per Sentence for each file: {average_words_per_sentence_list}")

Average Number of Words Per Sentence for each file: [34.3, 23.666666666666668, 23.666666666666668, 24.333333333333332, 23.666666666666668, 23.333333333333332, 23.666666666666668, 23.333333333333332, 26.666666666666668, 25.333333333333332, 24.666666666666668, 16.61764705882353, 22.352941176470587, 27.0, 18.516129032258064, 22.045454545454547, 19.90909090909091, 19.967741935483872, 22.625, 18.161290322580644, 19.0625, 24.555555555555557, 16.130434782608695, 20.666666666666668, 18.08695652173913, 16.86842105263158, 33.12, 24.153846153846153, 18.916666666666668, 88.0, 34.0, 50.666666666666664, 67.0, 13.44, 21.083333333333332, 16.666666666666668, 42.2, 34.333333333333336, 27.8, 28.5, 34.77777777777778, 21.181818181818183, 23.454545454545453, 16.130434782608695, 15.538461538461538, 15.954545454545455, 25.77777777777778, 55.0, 20.0, 27.0, 24.375, 21.714285714285715, 18.166666666666668, 17.65, 27.846153846153847, 28.428571428571427, 26.214285714285715, 17.272727272727273, 16.2, 19.6, 23.6, 19.

In [ ]:
complex_word_counts = []
for file_path in file_paths:
    text = read_text(file_path)
    complex_word_count = 0
    words = word_tokenize(text)
    for word in words:
        if word.isalpha() and syllable_count(word) > 2:
            complex_word_count += 1
    complex_word_counts.append(complex_word_count)

print(f"Number of Complex Words for each file: {complex_word_counts}")

Number of Complex Words for each file: [203, 17, 21, 21, 22, 20, 18, 19, 23, 21, 23, 123, 89, 44, 115, 121, 105, 134, 131, 117, 164, 41, 111, 78, 120, 135, 125, 41, 36, 22, 12, 22, 28, 65, 46, 26, 51, 27, 66, 47, 64, 29, 39, 34, 49, 80, 38, 20, 19, 26, 48, 37, 18, 61, 76, 40, 54, 64, 29, 17, 26, 27, 26, 39, 50, 73, 61, 56, 49, 52, 189, 29, 62, 8, 52, 43, 24, 40, 16, 62, 8, 29, 44, 48, 31, 15, 37, 51, 28, 12, 26, 145, 140, 16, 36, 30, 37, 43, 75, 32, 43, 23, 43, 5, 83, 30, 44, 18, 29, 44, 59, 33, 45, 10, 34, 13, 53, 48, 4, 18, 20, 9, 17, 11, 43, 22, 76, 29, 74, 37, 47, 44, 24, 23, 22, 25, 21, 18, 19, 24, 136, 118, 171, 226, 23, 6, 21]


In [ ]:
word_counts = []
for file_path in file_paths:
    text = read_text(file_path)
    tokens = tokenize(text)
    word_count = len(tokens)
    word_counts.append(word_count)

print(f"Cleaned Number of Words for each file: {word_counts}")

Cleaned Number of Words for each file: [469, 55, 53, 55, 55, 52, 53, 55, 63, 55, 60, 383, 262, 146, 373, 320, 281, 409, 352, 370, 413, 141, 267, 205, 274, 399, 491, 179, 140, 116, 50, 96, 157, 231, 162, 108, 154, 124, 252, 172, 199, 151, 173, 205, 214, 246, 157, 82, 69, 77, 145, 191, 88, 204, 222, 133, 209, 226, 108, 68, 81, 84, 72, 112, 189, 224, 247, 283, 236, 188, 476, 156, 215, 53, 185, 170, 78, 110, 165, 185, 50, 113, 161, 201, 140, 61, 114, 171, 115, 94, 134, 467, 485, 49, 165, 106, 171, 137, 206, 134, 171, 70, 162, 19, 362, 205, 179, 93, 167, 196, 283, 166, 205, 49, 113, 81, 196, 163, 22, 65, 72, 52, 55, 51, 179, 86, 240, 128, 207, 164, 184, 170, 99, 94, 87, 94, 89, 75, 73, 106, 557, 277, 529, 834, 120, 17, 74]


In [ ]:
average_syllables_per_word_list = []
for file_path in file_paths:
    text = read_text(file_path)
    words = word_tokenize(text)
    syllable_per_word = [syllable_count(word) for word in words]
    average_syllables = stat.mean(syllable_per_word) if syllable_per_word else 0 # Handle empty lists
    average_syllables_per_word_list.append(average_syllables)

print(f"Average Syllables per Word for each file: {average_syllables_per_word_list}")

Average Syllables per Word for each file: [1.9025270758122743, 1.7592592592592593, 1.8037383177570094, 1.7767857142857142, 1.8256880733944953, 1.766355140186916, 1.7383177570093458, 1.7850467289719627, 1.8264462809917354, 1.8275862068965518, 1.8198198198198199, 1.7963224893917964, 1.7516059957173447, 1.7127272727272727, 1.68475073313783, 1.748299319727891, 1.730188679245283, 1.758108108108108, 1.7546583850931676, 1.7463556851311954, 1.81651376146789, 1.5923344947735192, 1.9061833688699361, 1.7866323907455013, 1.8265306122448979, 1.6842105263157894, 1.5128205128205128, 1.5182291666666667, 1.5444839857651245, 1.5666666666666667, 1.5326086956521738, 1.575268817204301, 1.4654545454545456, 1.7142857142857142, 1.6360759493670887, 1.6772486772486772, 1.6954887218045114, 1.5083333333333333, 1.5735607675906182, 1.5771604938271604, 1.6997389033942558, 1.4875444839857652, 1.6695906432748537, 1.4262295081967213, 1.459915611814346, 1.6595238095238096, 1.5880281690140845, 1.5973154362416107, 1.69465

In [ ]:
def personal_pronouns(text):
    pronouns = ['i', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours']
    count = 0
    text = text.replace('US', '').strip()  # Remove 'US' and strip whitespace
    for word in text.lower().split():
        if word in pronouns:
            count += 1
    return count

personal_pronoun_counts = []
for file_path in file_paths:
    text = read_text(file_path)
    count = personal_pronouns(text)
    personal_pronoun_counts.append(count)

print(f"Personal Pronoun Counts for each file: {personal_pronoun_counts}")

Personal Pronoun Counts for each file: [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 2, 34, 15, 11, 1, 29, 6, 14, 4, 2, 1, 6, 1, 10, 6, 5, 5, 1, 4, 1, 6, 4, 3, 1, 3, 11, 4, 2, 4, 10, 20, 8, 2, 1, 1, 2, 1, 1, 10, 1, 8, 4, 3, 14, 10, 1, 2, 2, 3, 2, 4, 8, 17, 12, 13, 17, 9, 7, 3, 1, 1, 5, 6, 2, 5, 11, 0, 0, 1, 1, 0, 0, 2, 2, 3, 2, 3, 2, 3, 1, 0, 4, 2, 3, 4, 4, 8, 8, 2, 7, 0, 4, 5, 0, 1, 8, 7, 8, 10, 5, 3, 1, 1, 2, 3, 0, 0, 0, 0, 0, 0, 8, 4, 3, 2, 16, 8, 1, 8, 0, 0, 0, 0, 0, 0, 0, 1, 7, 3, 4, 6, 11, 0, 1]


In [ ]:
average_word_lengths = []
for file_path in file_paths:
    text = read_text(file_path)
    words = word_tokenize(text)  # Assuming you have the word_tokenize function available
    num_words = len(words)
    average_word_length = sum(len(word) for word in words) / num_words if num_words > 0 else 0
    average_word_lengths.append(average_word_length)

print(f"Average Word Lengths for each file: {average_word_lengths}")

Average Word Lengths for each file: [5.580024067388688, 5.407407407407407, 5.4953271028037385, 5.303571428571429, 5.495412844036697, 5.392523364485982, 5.383177570093458, 5.4953271028037385, 5.512396694214876, 5.655172413793103, 5.531531531531532, 5.548797736916549, 5.28051391862955, 5.210909090909091, 4.978005865102639, 5.224489795918367, 5.1471698113207545, 5.241891891891892, 5.208074534161491, 5.145772594752186, 5.4036697247706424, 4.637630662020906, 5.6034115138592755, 5.244215938303342, 5.073469387755102, 4.859649122807017, 4.542735042735043, 4.619791666666667, 4.4661921708185055, 5.109523809523809, 4.6521739130434785, 4.645161290322581, 4.305454545454546, 5.338155515370706, 4.9905063291139244, 5.031746031746032, 4.943609022556391, 4.65, 4.62046908315565, 4.811728395061729, 4.932114882506528, 4.412811387900356, 5.295321637426901, 4.203747072599532, 4.419831223628692, 4.780952380952381, 4.813380281690141, 4.953020134228188, 5.442748091603053, 4.513333333333334, 4.754716981132075, 4

In [ ]:
col=['POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE',"COMPLEX WORD COUNT","WORD COUNT","SYLLABLE PER WORD","PERSONAL PRONOUNS",'AVG WORD LENGTH']
val=[positive_scores,negative_scores,polarity_score,subjectivity_score,average_sentence_lengths,percentage_complex_words_list,fog_indices,average_words_per_sentence_list,complex_word_counts,word_counts,average_syllables_per_word_list,personal_pronoun_counts,average_word_lengths]
for i in range(len(col)):
  df[col[i]]=val[i]
df


,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,32,8,0.600000,0.085288,34.300000,0.295918,13.838367,34.300000,203,469,1.902527,2,5.580024
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,2,0,1.000000,0.036364,23.666667,0.239437,9.562441,23.666667,17,55,1.759259,1,5.407407
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,2,0,1.000000,0.037736,23.666667,0.295775,9.584977,23.666667,21,53,1.803738,1,5.495327
3,bctech2014,https://insights.blackcoffer.com/effective-man...,2,0,1.000000,0.036364,24.333333,0.287671,9.848402,24.333333,21,55,1.776786,1,5.303571
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,3,0,1.000000,0.054545,23.666667,0.309859,9.590610,23.666667,22,55,1.825688,1,5.495413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,bctech2153,https://insights.blackcoffer.com/population-an...,14,9,0.217391,0.043478,23.235294,0.216456,9.380700,23.235294,171,529,1.721573,4,4.829968
143,bctech2154,https://insights.blackcoffer.com/google-lsa-ap...,22,19,0.073171,0.049161,20.203125,0.174787,8.151165,20.203125,226,834,1.664439,6,4.794786
144,bctech2155,https://insights.blackcoffer.com/healthcare-da...,8,7,0.066667,0.125000,19.909091,0.105023,8.005645,19.909091,23,120,1.500000,11,4.389313
145,bctech2156,https://insights.blackcoffer.com/budget-sales-...,0,0,0.000000,0.000000,17.000000,0.352941,6.941176,17.000000,6,17,2.736842,0,8.105263


In [ ]:
output_excel_path = 'text_analysis_results.xlsx'
df.to_excel(output_excel_path, index=False)

print(f"Results saved to {output_excel_path}")

Results saved to text_analysis_results.xlsx
